In [ ]:
# get the data(Project Gutenberg eBook of No title)

# !wget https://www.gutenberg.org/cache/epub/71695/pg71695.txt -O no_title_book.txt

--2023-09-23 05:25:36--  https://www.gutenberg.org/cache/epub/71695/pg71695.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 554094 (541K) [text/plain]
Saving to: ‘no_title_book.txt’

no_title_book.txt   100%[===================>] 541.11K  2.66MB/s    in 0.2s    

2023-09-23 05:25:36 (2.66 MB/s) - ‘no_title_book.txt’ saved [554094/554094]



In [ ]:
text = open("no_title_book.txt", 'r', encoding="utf-8").read()
text = text.lower()

In [ ]:
sentences = text.split("\n")

In [ ]:
# Few lines of the book
few_lines = [sentences[i] for i in range(6)]
few_lines

['\ufeffthe project gutenberg ebook of no title',
 '    ',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online']

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(oov_token = "<UNK>" )

In [ ]:
tokenizer.fit_on_texts(sentences)

In [ ]:
vocab_size = len(tokenizer.word_index)
vocab_size

8569

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
input_sequences = []

for sequence in sequences:
  for i in range(1, len(sequence)):
    n_gram_seq = sequence[:i+1]
    input_sequences.append(n_gram_seq)

In [ ]:
input_sequences[:2]

[[4663, 132], [4663, 132, 292]]

In [ ]:
#  we need sequences of the same length so we're padding it.

max_seq_len = max([len(seq)for seq in input_sequences])
max_seq_len

29

In [ ]:
padded_seqs = pad_sequences(input_sequences, maxlen = max_seq_len, padding="pre")

In [ ]:
padded_seqs[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 4663,  132],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 4663,  132,  292]], dtype=int32)

We're going to form input-output sequence

In [ ]:
padded_seqs = np.array(padded_seqs)

In [ ]:
X = padded_seqs[:, :-1]
labels = padded_seqs[:, -1]

In [ ]:
len(labels)

85860

In [ ]:
X.shape

(85860, 28)

In [ ]:
# lab = np.where(labels >= vocab_size, 0, labels)
# print(max(lab))
# max([l for l in labels])

In [ ]:
y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size+1)

In [ ]:
print("The Shape of X is ", X.shape)
print("The Shape of y is ", y.shape)

The Shape of X is  (85860, 28)
The Shape of y is  (85860, 8570)


In [ ]:
# Create Model

from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Bidirectional

In [ ]:
vocab_size

8569

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size+1, 100,input_length= max_seq_len-1))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(vocab_size+1, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 100)           857000    
                                                                 
 bidirectional (Bidirection  (None, 1024)              2510848   
 al)                                                             
                                                                 
 dense (Dense)               (None, 8570)              8784250   
                                                                 
Total params: 12152098 (46.36 MB)
Trainable params: 12152098 (46.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# from keras.callbacks import EarlyStopping
# es = EarlyStopping(monitor="acc", min_delta=0.01)

In [ ]:
model.fit(X,y, epochs = 30, batch_size=256, verbose=2)

Epoch 1/14
336/336 - 18s - loss: 0.6134 - accuracy: 0.8840 - 18s/epoch - 54ms/step
Epoch 2/14
336/336 - 18s - loss: 0.5703 - accuracy: 0.8902 - 18s/epoch - 53ms/step
Epoch 3/14
336/336 - 17s - loss: 0.5363 - accuracy: 0.8946 - 17s/epoch - 51ms/step
Epoch 4/14
336/336 - 17s - loss: 0.5063 - accuracy: 0.8990 - 17s/epoch - 50ms/step
Epoch 5/14
336/336 - 17s - loss: 0.4823 - accuracy: 0.9010 - 17s/epoch - 50ms/step
Epoch 6/14
336/336 - 17s - loss: 0.4640 - accuracy: 0.9036 - 17s/epoch - 51ms/step
Epoch 7/14
336/336 - 17s - loss: 0.4467 - accuracy: 0.9053 - 17s/epoch - 51ms/step
Epoch 8/14
336/336 - 18s - loss: 0.4287 - accuracy: 0.9076 - 18s/epoch - 54ms/step
Epoch 9/14
336/336 - 17s - loss: 0.4205 - accuracy: 0.9080 - 17s/epoch - 51ms/step
Epoch 10/14
336/336 - 17s - loss: 0.4106 - accuracy: 0.9092 - 17s/epoch - 51ms/step
Epoch 11/14
336/336 - 17s - loss: 0.3995 - accuracy: 0.9101 - 17s/epoch - 51ms/step
Epoch 12/14
336/336 - 17s - loss: 0.3864 - accuracy: 0.9115 - 17s/epoch - 50ms/step
E

In [ ]:
#example_text =  This by his voice, should
# generated_text should be "be a Montague."  (if lenght = 3)

example_text = "In view of this very condensed account of the remarkable life and work of the children in this household"
length = 50

for _ in range(length):

  token_example_text = tokenizer.texts_to_sequences([example_text])
  padded_token_text = pad_sequences(token_example_text, maxlen = max_seq_len-1)
  predicted_word_pos = np.argmax(model.predict(padded_token_text, verbose = 0))

  output_sentence = " "

  for word, index in tokenizer.word_index.items():
    if index == predicted_word_pos:
      output_sentence = word
      example_text += " " + output_sentence
      break
print(example_text)

In view of this very condensed account of the remarkable life and work of the children in this household not well that the man had he had to have to wait his trust of his work concerning it in the face of peking to have some of these letters as the school which he did not seem to be a missionary but i looked him in a more type
